In [1]:
import pandas as pd
import numpy as np

In [33]:
df = pd.read_csv('FinalBikeShareformodelling.csv')

In [34]:
df.head(2)

,duration,holiday,day_of_week,member,season,month_year,year,description,avg_temp,start_date_time
0,2389,1,3,0,1,2015-01,2015,Clear,-1.0,2015-01-01 00:02:44
1,2394,1,3,0,1,2015-01,2015,Clear,-1.0,2015-01-01 00:02:46


### DataFrame grouped by date, hour and membertype

In [35]:
#Converted to datetime column to extract hourly information
#df['date']= pd.to_datetime(df['start_date_time'])

df['hour'] =df['date'].dt.hour

In [47]:
df['date'] = df['date'].dt.date

In [48]:
print(df.shape)
df.head()

(10277677, 11)


,duration,holiday,day_of_week,member,season,month_year,year,description,avg_temp,date,hour
0,2389,1,3,0,1,2015-01,2015,Clear,-1.0,2015-01-01,0
1,2394,1,3,0,1,2015-01,2015,Clear,-1.0,2015-01-01,0
2,468,1,3,1,1,2015-01,2015,Clear,-1.0,2015-01-01,0
3,348,1,3,1,1,2015-01,2015,Clear,-1.0,2015-01-01,0
4,980,1,3,0,1,2015-01,2015,Clear,-1.0,2015-01-01,0


In [37]:
df.drop(['start_date_time'], axis=1, inplace=True)

In [49]:
df_2017 = pd.DataFrame(df.groupby([ "date", "hour",'member']).agg({'duration':'count'})).reset_index()

In [50]:
print(df_2017.shape)
df_2017.head()

(50407, 4)


,date,hour,member,duration
0,2015-01-01,0,0,11
1,2015-01-01,0,1,43
2,2015-01-01,1,0,9
3,2015-01-01,1,1,105
4,2015-01-01,2,0,5


In [51]:
df_2018 = pd.read_csv('hour_grouped.csv')

In [52]:
df_2018.head()

,date,hour,Duration,Member
0,2018-01-01,0.0,6,0
1,2018-01-01,0.0,28,1
2,2018-01-01,1.0,11,0
3,2018-01-01,1.0,38,1
4,2018-01-01,2.0,7,0


In [56]:
#df_2018.head()
df_2018.columns = ['date', 'hour','member','duration']

In [59]:
df_2018.head(2)

,date,hour,member,duration
0,2018-01-01,0.0,6,0
1,2018-01-01,0.0,28,1


### Two dataframes concatenated

In [118]:
df = pd.concat([df_2017, df_2018], axis=0, ignore_index=True)

In [119]:
df.shape

(64734, 4)

In [120]:
df.head()

,date,hour,member,duration
0,2015-01-01,0.0,0,11
1,2015-01-01,0.0,1,43
2,2015-01-01,1.0,0,9
3,2015-01-01,1.0,1,105
4,2015-01-01,2.0,0,5


In [121]:
#changing column names
df.columns =['date', 'hour', 'member', 'count']

In [122]:
#extracting month information from the datetime column
df['date']= pd.to_datetime(df['date'])
df['month'] = df['date'].dt.month

In [123]:
df.shape

(64734, 5)

In [124]:
df.head()

,date,hour,member,count,month
0,2015-01-01,0.0,0,11,1
1,2015-01-01,0.0,1,43,1
2,2015-01-01,1.0,0,9,1
3,2015-01-01,1.0,1,105,1
4,2015-01-01,2.0,0,5,1


In [125]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64734 entries, 0 to 64733
Data columns (total 5 columns):
date      64734 non-null datetime64[ns]
hour      64734 non-null float64
member    64734 non-null int64
count     64734 non-null int64
month     64734 non-null int64
dtypes: datetime64[ns](1), float64(1), int64(3)
memory usage: 2.5 MB


### Adding in new feature
#### 1.Holiday

In [126]:
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar
import holidays

us_holidays = holidays.UnitedStates(years=[2015,2016,2017,2018])

df['holiday'] = df['date'].isin(us_holidays)

In [127]:
df['holiday'] = df.holiday.astype(int)

#### 2. Weekday

In [128]:
#New feature: Adding in the day of the weeek
df['day_of_week'] = df['date'].dt.day_name()

#### 3. Season

In [129]:
seasons = ['winter', 'winter','spring','spring','spring', 'summer', 'summer', 'summer', 'autumn', 'autumn', 'autumn','winter']

month_to_season = dict(zip(range(1,13), seasons))

month_to_season 

{1: 'winter',
 2: 'winter',
 3: 'spring',
 4: 'spring',
 5: 'spring',
 6: 'summer',
 7: 'summer',
 8: 'summer',
 9: 'autumn',
 10: 'autumn',
 11: 'autumn',
 12: 'winter'}

In [130]:
#adding a new column called season and then delete the month column
df['season']= df['month'].replace(month_to_season)

In [131]:
df.drop(['month'], axis=1, inplace=True)

In [132]:
df.shape

(64734, 7)

In [133]:
df.head()

,date,hour,member,count,holiday,day_of_week,season
0,2015-01-01,0.0,0,11,1,Thursday,winter
1,2015-01-01,0.0,1,43,1,Thursday,winter
2,2015-01-01,1.0,0,9,1,Thursday,winter
3,2015-01-01,1.0,1,105,1,Thursday,winter
4,2015-01-01,2.0,0,5,1,Thursday,winter


### Adding in scraped weather information

In [90]:
weather_df = pd.read_csv('cleaned_weather_df.csv')

In [91]:
#weather description values replaced
weather_df['Description'] = weather_df.Description.replace('weather events not reported','Clear')
weather_df['Description'] = weather_df.Description.replace('0.0','Clear')

In [92]:
weather_df = weather_df.drop(['Daily minimum temperature','Daily maximum temperature'], axis=1)

In [178]:
weather_df.tail()

,Date,Description,avg_temp
1091,2017-12-27,Clear,-1.0
1092,2017-12-28,Clear,-6.5
1093,2017-12-29,Clear,-6.5
1094,2017-12-30,Clear,-3.5
1095,2017-12-31,Clear,-4.5


In [179]:
weather_df = pd.read_csv('NewWeatherfile.csv')

In [180]:
weather_df['Description'] = weather_df.Description.replace('0.0','Clear')

In [98]:
weather_df.Description.value_counts()
weather_df.columns = ['date', 'description', 'avg_temp']

In [99]:
weather_df.shape

(1096, 3)

In [100]:
weather_df['date']= pd.to_datetime(weather_df['date'])

In [101]:
weather_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1096 entries, 0 to 1095
Data columns (total 3 columns):
date           1096 non-null datetime64[ns]
description    1096 non-null object
avg_temp       1096 non-null float64
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 25.8+ KB


### Merging weather dataframe and original dataframe

In [102]:
df = pd.merge(df, weather_df, how='left', on = 'date')

In [105]:
df.head()

,date,hour,member,count,holiday,day_of_week,season,description,avg_temp
0,2015-01-01,0.0,0,11,1,Thursday,winter,Clear,-1.0
1,2015-01-01,0.0,1,43,1,Thursday,winter,Clear,-1.0
2,2015-01-01,1.0,0,9,1,Thursday,winter,Clear,-1.0
3,2015-01-01,1.0,1,105,1,Thursday,winter,Clear,-1.0
4,2015-01-01,2.0,0,5,1,Thursday,winter,Clear,-1.0


In [104]:
#checking for any null values
df.isnull().sum()

date               0
hour               0
member             0
count              0
holiday            0
day_of_week        0
season             0
description    14327
avg_temp       14327
dtype: int64

In [106]:
df[df.avg_temp.isnull()]

,date,hour,member,count,holiday,day_of_week,season,description,avg_temp
50407,2018-01-01,0.0,6,0,1,Monday,winter,NaN,NaN
50408,2018-01-01,0.0,28,1,1,Monday,winter,NaN,NaN
50409,2018-01-01,1.0,11,0,1,Monday,winter,NaN,NaN
50410,2018-01-01,1.0,38,1,1,Monday,winter,NaN,NaN
50411,2018-01-01,2.0,7,0,1,Monday,winter,NaN,NaN
50412,2018-01-01,2.0,30,1,1,Monday,winter,NaN,NaN
50413,2018-01-01,3.0,1,0,1,Monday,winter,NaN,NaN
50414,2018-01-01,3.0,8,1,1,Monday,winter,NaN,NaN
50415,2018-01-01,4.0,2,0,1,Monday,winter,NaN,NaN
50416,2018-01-01,4.0,10,1,1,Monday,winter,NaN,NaN


In [108]:
#Missing weather information for two dates
print(df[df.date == '2018-12-30'].index)
print(df[df.date == '2018-12-30'].index)

Int64Index([64642, 64643, 64644, 64645, 64646, 64647, 64648, 64649, 64650,
            64651, 64652, 64653, 64654, 64655, 64656, 64657, 64658, 64659,
            64660, 64661, 64662, 64663, 64664, 64665, 64666, 64667, 64668,
            64669, 64670, 64671, 64672, 64673, 64674, 64675, 64676, 64677,
            64678, 64679, 64680, 64681, 64682, 64683, 64684, 64685, 64686,
            64687, 64688],
           dtype='int64')
Int64Index([64642, 64643, 64644, 64645, 64646, 64647, 64648, 64649, 64650,
            64651, 64652, 64653, 64654, 64655, 64656, 64657, 64658, 64659,
            64660, 64661, 64662, 64663, 64664, 64665, 64666, 64667, 64668,
            64669, 64670, 64671, 64672, 64673, 64674, 64675, 64676, 64677,
            64678, 64679, 64680, 64681, 64682, 64683, 64684, 64685, 64686,
            64687, 64688],
           dtype='int64')


### Imputing missing weather information

In [109]:
df.loc[64689: 64733,'avg_temp'] = 6.0
df.loc[64689: 64733,'description'] = 'Rain'

In [110]:
df.loc[64642:64688,'avg_temp'] = 5.0
df.loc[64642:64688,'description'] = 'Clear'

In [112]:
df.tail()

,date,hour,member,count,holiday,day_of_week,season,description,avg_temp
64729,2018-12-31,21.0,58,1,0,Monday,winter,Rain,6.0
64730,2018-12-31,22.0,4,0,0,Monday,winter,Rain,6.0
64731,2018-12-31,22.0,46,1,0,Monday,winter,Rain,6.0
64732,2018-12-31,23.0,6,0,0,Monday,winter,Rain,6.0
64733,2018-12-31,23.0,25,1,0,Monday,winter,Rain,6.0


### Categorising hour information into different time 

In [113]:
def daily_time(hour):
    if hour <= 3 or hour >= 23:
        return 'midnight'
    elif hour > 3 and hour <= 6:
        return 'early morning'
    elif hour > 6 and hour <= 9:
        return 'morning'
    elif hour > 9 and hour <= 11:
        return 'late morning'
    elif hour > 11 and hour <= 15:
        return 'afternoon'
    elif hour > 15 and hour <= 19:
        return 'evening'
    else:
        return 'night'

In [114]:
df['time'] = df.hour.apply(lambda x : daily_time(x))

In [115]:
df.head()

,date,hour,member,count,holiday,day_of_week,season,description,avg_temp,time
0,2015-01-01,0.0,0,11,1,Thursday,winter,Clear,-1.0,midnight
1,2015-01-01,0.0,1,43,1,Thursday,winter,Clear,-1.0,midnight
2,2015-01-01,1.0,0,9,1,Thursday,winter,Clear,-1.0,midnight
3,2015-01-01,1.0,1,105,1,Thursday,winter,Clear,-1.0,midnight
4,2015-01-01,2.0,0,5,1,Thursday,winter,Clear,-1.0,midnight


In [116]:
df.tail()

,date,hour,member,count,holiday,day_of_week,season,description,avg_temp,time
64729,2018-12-31,21.0,58,1,0,Monday,winter,Rain,6.0,night
64730,2018-12-31,22.0,4,0,0,Monday,winter,Rain,6.0,night
64731,2018-12-31,22.0,46,1,0,Monday,winter,Rain,6.0,night
64732,2018-12-31,23.0,6,0,0,Monday,winter,Rain,6.0,midnight
64733,2018-12-31,23.0,25,1,0,Monday,winter,Rain,6.0,midnight


In [117]:
df.time.value_counts()

midnight         12763
afternoon        11158
evening          11156
morning           8328
night             8318
early morning     7435
late morning      5576
Name: time, dtype: int64

### We now have a dataframe ready to model with all the important features

In [227]:
df.to_csv('RegressionModelFile2018.csv', index=False)